In [2]:
%load_ext heir_play

Loading heir-opt nightly binary
Using existing local heir-opt
Loading heir-translate nightly binary


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 14.1M  100 14.1M    0     0  3355k      0  0:00:04  0:00:04 --:--:-- 3712k


In [3]:
%%heir_opt --convert-if-to-select --canonicalize

func.func @secret_condition_with_non_secret_int(%inp: i16, %cond: !secret.secret<i1>) -> !secret.secret<i16> {
  %0 = secret.generic ins(%inp, %cond : i16, !secret.secret<i1>) {
  ^bb0(%copy_inp: i16, %secret_cond: i1):
    %1 = scf.if %secret_cond -> (i16) {
      %2 = arith.addi %copy_inp, %copy_inp : i16
      scf.yield %2 : i16
    } else {
      scf.yield %copy_inp : i16
    }
    secret.yield %1 : i16
  } -> !secret.secret<i16>
  return %0 : !secret.secret<i16>
}

Running heir-opt...
module {
  func.func @secret_condition_with_non_secret_int(%arg0: i16, %arg1: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = arith.addi %arg0, %arg0 : i16
    %1 = secret.generic ins(%arg1 : !secret.secret<i1>) {
    ^bb0(%arg2: i1):
      %2 = arith.select %arg2, %0, %arg0 : i16
      secret.yield %2 : i16
    } -> !secret.secret<i16>
    return %1 : !secret.secret<i16>
  }
}

None


Operation: %2 = arith.addi %arg2, %arg2 : i16
	Operand : <block argument> of type 'i16' at index: 0; Secretness: 0
	Operand : <block argument> of type 'i16' at index: 0; Secretness: 0
	Result : %2 = arith.addi %arg2, %arg2 : i16; Secretness: 0
Operation: scf.yield %2 : i16
	Operand : %2 = arith.addi %arg2, %arg2 : i16; Secretness: 0
Operation: scf.yield %arg2 : i16
	Operand : <block argument> of type 'i16' at index: 0; Secretness: 0
Operation: %1 = scf.if %arg3 -> (i16) {
  %2 = arith.addi %arg2, %arg2 : i16
  scf.yield %2 : i16
} else {
  scf.yield %arg2 : i16
}
	Operand : <block argument> of type 'i1' at index: 1; Secretness: 1
	Result : %1 = scf.if %arg3 -> (i16) {
  %2 = arith.addi %arg2, %arg2 : i16
  scf.yield %2 : i16
} else {
  scf.yield %arg2 : i16
}; Secretness: 0
Operation: secret.yield %1 : i16
	Operand : %1 = scf.if %arg3 -> (i16) {
  %2 = arith.addi %arg2, %arg2 : i16
  scf.yield %2 : i16
} else {
  scf.yield %arg2 : i16
}; Secretness: 0
Operation: %0 = secret.generic ins

In [6]:
%%heir_translate --emit-openfhe-pke

#degree_32_poly = #polynomial.int_polynomial<1 + x**32>
#eval_encoding = #lwe.polynomial_evaluation_encoding<cleartext_start = 16, cleartext_bitwidth = 16>
#ring2 = #polynomial.ring<coefficientType = i32, polynomialModulus=#degree_32_poly>
#params2 = #lwe.rlwe_params<ring = #ring2>
!tensor_pt_ty = !lwe.rlwe_plaintext<encoding = #eval_encoding, ring = #ring2, underlying_type = tensor<32xi16>>
!scalar_pt_ty = !lwe.rlwe_plaintext<encoding = #eval_encoding, ring = #ring2, underlying_type = i16>
!tensor_ct_ty = !lwe.rlwe_ciphertext<encoding = #eval_encoding, rlwe_params = #params2, underlying_type = tensor<32xi16>>
!scalar_ct_ty = !lwe.rlwe_ciphertext<encoding = #eval_encoding, rlwe_params = #params2, underlying_type = i16>

func.func @simple_sum(%arg0: !openfhe.crypto_context, %arg1: !tensor_ct_ty) -> !scalar_ct_ty {
  %1 = openfhe.rot %arg0, %arg1 { index = 16 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %2 = openfhe.add %arg0, %arg1, %1 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_ct_ty) -> !tensor_ct_ty
  %4 = openfhe.rot %arg0, %2 { index = 8 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %5 = openfhe.add %arg0, %2, %4 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_ct_ty) -> !tensor_ct_ty
  %7 = openfhe.rot %arg0, %5 { index = 4 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %8 = openfhe.add %arg0, %5, %7 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_ct_ty) -> !tensor_ct_ty
  %10 = openfhe.rot %arg0, %8 { index = 2 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %11 = openfhe.add %arg0, %8, %10 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_ct_ty) -> !tensor_ct_ty
  %13 = openfhe.rot %arg0, %11 { index = 1 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %14 = openfhe.add %arg0, %11, %13 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_ct_ty) -> !tensor_ct_ty
  %cst = arith.constant dense<[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]> : tensor<32xi16>
  %15 = lwe.rlwe_encode %cst {encoding = #eval_encoding, ring = #ring2} : tensor<32xi16> -> !tensor_pt_ty
  %16 = openfhe.mul_plain %arg0, %14, %15 : (!openfhe.crypto_context, !tensor_ct_ty, !tensor_pt_ty) -> !tensor_ct_ty
  %18 = openfhe.rot %arg0, %16 { index = 31 } : (!openfhe.crypto_context, !tensor_ct_ty) -> !tensor_ct_ty
  %19 = lwe.reinterpret_application_data %18 : !tensor_ct_ty to !scalar_ct_ty
  return %19 : !scalar_ct_ty
}
func.func @simple_sum__encrypt(%arg0: !openfhe.crypto_context, %arg1: tensor<32xi16>, %arg2: !openfhe.public_key) -> !tensor_ct_ty {
  %0 = lwe.rlwe_encode %arg1 {encoding = #eval_encoding, ring = #ring2} : tensor<32xi16> -> !tensor_pt_ty
  %1 = openfhe.encrypt %arg0, %0, %arg2 : (!openfhe.crypto_context, !tensor_pt_ty, !openfhe.public_key) -> !tensor_ct_ty
  return %1 : !tensor_ct_ty
}
func.func @simple_sum__decrypt(%arg0: !openfhe.crypto_context, %arg1: !scalar_ct_ty, %arg2: !openfhe.private_key) -> i16 {
  %0 = openfhe.decrypt %arg0, %arg1, %arg2 : (!openfhe.crypto_context, !scalar_ct_ty, !openfhe.private_key) -> !scalar_pt_ty
  %1 = lwe.rlwe_decode %0 {encoding = #eval_encoding, ring = #ring2} : !scalar_pt_ty -> i16
  return %1 : i16
}

Running heir-translate...

#include "src/pke/include/openfhe.h" // from @openfhe

using namespace lbcrypto;
using CiphertextT = ConstCiphertext<DCRTPoly>;
using CCParamsT = CCParams<CryptoContextBGVRNS>;
using CryptoContextT = CryptoContext<DCRTPoly>;
using EvalKeyT = EvalKey<DCRTPoly>;
using PlaintextT = Plaintext;
using PrivateKeyT = PrivateKey<DCRTPoly>;
using PublicKeyT = PublicKey<DCRTPoly>;

CiphertextT simple_sum(CryptoContextT v0, CiphertextT v1) {
  const auto& v2 = v0->EvalRotate(v1, 16);
  const auto& v3 = v0->EvalAdd(v1, v2);
  const auto& v4 = v0->EvalRotate(v3, 8);
  const auto& v5 = v0->EvalAdd(v3, v4);
  const auto& v6 = v0->EvalRotate(v5, 4);
  const auto& v7 = v0->EvalAdd(v5, v6);
  const auto& v8 = v0->EvalRotate(v7, 2);
  const auto& v9 = v0->EvalAdd(v7, v8);
  const auto& v10 = v0->EvalRotate(v9, 1);
  const auto& v11 = v0->EvalAdd(v9, v10);
  std::vector<int16_t> v12 = {0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1}